In [47]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob # for file handling

import googlemaps
import time
import dotenv as dotenv

### Section 1: Data concatenation, cleaning and preprocessing

In [32]:
# Preprocess initial .csv files from data.gov.sg
csv_files = glob.glob("./data/*.csv")
print(csv_files)
dataframes = [pd.read_csv(file) for file in csv_files]

for df in dataframes:
    df['month'] = pd.to_datetime(df['month'])

# check columns of each dataframe
for i, df in enumerate(dataframes):
    print(f"DataFrame {csv_files[i]} \nColumns: {df.columns}")

['./data\\Resale Flat Prices (Based on Approval Date), 1990 - 1999.csv', './data\\Resale Flat Prices (Based on Approval Date), 2000 - Feb 2012.csv', './data\\Resale Flat Prices (Based on Registration Date), From Jan 2015 to Dec 2016.csv', './data\\Resale Flat Prices (Based on Registration Date), From Mar 2012 to Dec 2014.csv', './data\\Resale flat prices based on registration date from Jan-2017 onwards.csv']
DataFrame ./data\Resale Flat Prices (Based on Approval Date), 1990 - 1999.csv 
Columns: Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price'],
      dtype='object')
DataFrame ./data\Resale Flat Prices (Based on Approval Date), 2000 - Feb 2012.csv 
Columns: Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price'],
      dtype='object')
DataFrame ./data\Resale Flat Prices (Based on Regist

### Decision to remove column 'remaining_lease' - that is onyl present for datasets post 2015
In order to maintain consistency between all datasets, we have decided to remove the remaining_lease column

In [ ]:
# Remove 'remaining_lease' column from all dataframes
for df in dataframes:
    if 'remaining_lease' in df.columns:
        df.drop(columns=['remaining_lease'], inplace=True)

# Create new data.csv file with all data combined
combined_df = pd.concat(dataframes, ignore_index=True)
combined_df.sort_values(by='month', inplace=True)

# count nan values in each column
print(combined_df.isna().sum())
# combined_df.dropna(inplace=True) # Not required as no nan values in the dataset

# due to nature of data, namely: storey range, dosent make sense to drop duplicates
# duplicates = combined_df[combined_df.duplicated()]
# print(f"Number of duplicate rows: {duplicates.shape[0]}")

# # remove duplicates
# combined_df.drop_duplicates(inplace=True)

# # check for duplicates again
# duplicates = combined_df[combined_df.duplicated()]
# print(f"Number of duplicate rows after removal: {duplicates.shape[0]}")

# reset index
combined_df.reset_index(drop=True, inplace=True)

# final csv file
# combined_df.to_csv("data.csv", index=False)

month                  0
town                   0
flat_type              0
block                  0
street_name            0
storey_range           0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
resale_price           0
dtype: int64


### Section 2: Feature Engineering: Calculating distance of HDB from MRT

In [52]:
# Using google maps api, get lat long coordinates of each location, add to dataframe

''' 
Rate Limit: 
3,000 QPM (queries per minute), calculated as the sum of client-side and server-side queries.

We have about 950k rows in our dataset, one way to solve would be to cache similar results
'''

# calculating number of similar blocks + street names to cache
count_pairs = combined_df.groupby(['block', 'street_name']).size().reset_index(name='counts')
print(count_pairs)

# drop 'counts' column
count_pairs.drop(columns=['counts'], inplace=True)

     block        street_name  counts
0        1           BEACH RD     147
1        1    BEDOK STH AVE 1     228
2        1       CHAI CHEE RD     129
3        1  CHANGI VILLAGE RD      35
4        1          DELTA AVE      89
...    ...                ...     ...
9885   99A    LOR 2 TOA PAYOH      50
9886   99B    LOR 2 TOA PAYOH      43
9887   99C    LOR 2 TOA PAYOH      52
9888    9A      BOON TIONG RD      32
9889    9B      BOON TIONG RD      26

[9890 rows x 3 columns]


##### Since we have only 10k unique combinations, we can just add a sleep (hopefully i dont go broke)

In [54]:
gmaps = googlemaps.Client(key=dotenv.get_key('.env', 'GOOGLE_MAPS_API_KEY'))

geocode_cache = {}

# test api call so i dont go broke
response = gmaps.geocode("10 Yishun Ave 9, Singapore")
print(response)
print(response[0]['geometry']['location'])


[{'address_components': [{'long_name': '10', 'short_name': '10', 'types': ['street_number']}, {'long_name': 'Yishun Avenue 9', 'short_name': 'Yishun Ave 9', 'types': ['route']}, {'long_name': 'Yishun', 'short_name': 'Yishun', 'types': ['neighborhood', 'political']}, {'long_name': 'Singapore', 'short_name': 'Singapore', 'types': ['locality', 'political']}, {'long_name': 'Singapore', 'short_name': 'SG', 'types': ['country', 'political']}, {'long_name': '768888', 'short_name': '768888', 'types': ['postal_code']}], 'formatted_address': '10 Yishun Ave 9, Singapore 768888', 'geometry': {'bounds': {'northeast': {'lat': 1.4318163, 'lng': 103.8400468}, 'southwest': {'lat': 1.43143, 'lng': 103.8396539}}, 'location': {'lat': 1.4316347, 'lng': 103.8398656}, 'location_type': 'ROOFTOP', 'viewport': {'northeast': {'lat': 1.432972130291502, 'lng': 103.8411993302915}, 'southwest': {'lat': 1.430274169708498, 'lng': 103.8385013697085}}}, 'navigation_points': [{'location': {'latitude': 1.4317223, 'longitu

In [55]:
def geocode_address(block, street):
    address = f"Block {block}, {street}, Singapore"
    try:
        geocode_result = gmaps.geocode(address)
        if geocode_result:
            location = geocode_result[0]['geometry']['location']
            return location['lat'], location['lng']
    except Exception as e:
        print(f"Error geocoding {address}: {e}")
    return None, None

# Loop through the DataFrame and geocode each unique address
count = 0
for idx, row in count_pairs.iterrows():
    block = row['block']
    street = row['street_name']
    
    # Skip if already cached
    if (block, street) in geocode_cache:
        continue
    
    # Call Geocoding API
    lat, lng = geocode_address(block, street)
    geocode_cache[(block, street)] = (lat, lng)
    print(f"Geocoded {block}, {street}: {lat}, {lng}")
    
    # Sleep every 2500 requests for 90s
    count += 1
    if count % 2500 == 0:
        print(f"Processed {count} addresses. Sleeping 90s for API limit")
        time.sleep(90)

# Convert cache dictionary to a DataFrame
cache_df = pd.DataFrame(
    [(k[0], k[1], v[0], v[1]) for k, v in geocode_cache.items()],
    columns=['block', 'street_name', 'latitude', 'longitude']
)

df_merged = combined_df.merge(cache_df, on=['block', 'street_name'], how='left')

# Save the result
df_merged.to_csv("data_with_latlong.csv", index=False)

Processed 2500 addresses. Sleeping 90s for API limit


KeyboardInterrupt: 